In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense,Dropout,Activation
from keras.optimizers import Adam
from keras.utils import np_utils

D:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
os.getcwd()

'D:\\Anaconda3\\Scripts\\1\\stock\\3029'

In [3]:
# #load all the price data
# years = ['2013','2014','2015','2016','2017','2018']
# df_all = pd.DataFrame()
# for year in years:
#     fn = '../rnn_data/all/'+year+'_all.csv'
#     d = pd.read_csv(fn,encoding='utf-8')
#     df_all = df_all.append(d)
# df_all.to_csv('../rnn_data/all/2013_2018_price.csv',encoding='utf-8')

In [4]:
df_all = pd.read_csv('../rnn_data/all/2013_2018_price.csv',encoding='utf-8')

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
#choose the stock based on stock code
df_c = df_all.groupby('證券代號').get_group('2337')

In [10]:
#delete some columns
del df_c['Unnamed: 0']
del df_c['最後揭示買價']
del df_c['最後揭示買量']
del df_c['最後揭示賣價']
del df_c['最後揭示賣量']
del df_c['證券代號']
del df_c['本益比']
d= df_c['日期']
del df_c['日期']
del df_c['成交金額']
del df_c['證券名稱']

In [11]:
df_c.columns

Index(['Unnamed: 0.1', '成交股數', '成交筆數', '開盤價', '最高價', '最低價', '收盤價', '漲跌(+/-)',
       '漲跌價差'],
      dtype='object')

In [12]:
del df_c['Unnamed: 0.1']

In [13]:
#drop the colume with missing value
empty_li = df_c[df_c['漲跌(+/-)']=='X'].index.values
for i in empty_li:
    df_c = df_c.drop(i)
empty_li = df_c[df_c['漲跌(+/-)']==' '].index.values
for i in empty_li:
    df_c = df_c.drop(i)

In [14]:
empty_li = df_c[df_c['收盤價']=='X'].index.values
for i in empty_li:
    df_c = df_c.drop(i)
empty_li = df_c[df_c['收盤價']==' '].index.values
for i in empty_li:
    df_c = df_c.drop(i)

In [15]:
#convert str to float and replace comma
df_c['成交筆數']=df_c['成交筆數'].str.replace(',','')
df_c['成交股數']=df_c['成交股數'].str.replace(',','')
df_c.成交筆數 = df_c.成交筆數.astype(float)
df_c.成交股數 = df_c.成交股數.astype(float)
df_c.收盤價 = df_c.收盤價.astype(float)
df_c.最低價 = df_c.最低價.astype(float)
df_c.最高價 = df_c.最高價.astype(float)
df_c.開盤價 = df_c.開盤價.astype(float)
df_c.漲跌價差 = df_c.漲跌價差.astype(float)

In [16]:
# #combine signal +/- and the difference of price 
df_c['return'] = np.where(df_c['漲跌(+/-)'] !='+',df_c['漲跌價差']*-1, df_c['漲跌價差'])
# del df_c['漲跌(+/-)']
# del df_c['漲跌價差']

In [17]:
df_c

,成交股數,成交筆數,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,return
290,15515039.0,3569.0,8.80,8.83,8.70,8.73,+,0.06,0.06
1161,31791025.0,6183.0,8.78,8.96,8.75,8.90,+,0.17,0.17
2032,20882138.0,4210.0,8.93,8.93,8.70,8.77,-,0.13,-0.13
2903,23474170.0,5669.0,8.77,9.07,8.66,8.97,+,0.20,0.20
3774,21819494.0,4274.0,8.84,8.84,8.70,8.75,-,0.22,-0.22
4645,28184977.0,7204.0,8.75,9.04,8.75,8.94,+,0.19,0.19
5516,25128836.0,5207.0,9.00,9.04,8.80,8.80,-,0.14,-0.14
6387,11023444.0,2643.0,8.96,9.02,8.88,8.90,+,0.10,0.10
7258,24175523.0,4853.0,8.93,9.06,8.82,9.02,+,0.12,0.12
8129,14471305.0,3988.0,9.02,9.06,8.87,8.89,-,0.13,-0.13


In [18]:
df_c['return']=df_c['return'].shift(-1)

In [19]:
# 5days average
df_c['label'] = (df_c['return']+df_c['return'].shift(-4)+df_c['return'].shift(-3)+df_c['return'].shift(-2)+df_c['return'].shift(-1))/5/df_c['收盤價']*100

In [20]:
df_c

,成交股數,成交筆數,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,return,label
290,15515039.0,3569.0,8.80,8.83,8.70,8.73,+,0.06,0.17,0.481100
1161,31791025.0,6183.0,8.78,8.96,8.75,8.90,+,0.17,-0.13,-0.224719
2032,20882138.0,4210.0,8.93,8.93,8.70,8.77,-,0.13,0.20,0.296465
2903,23474170.0,5669.0,8.77,9.07,8.66,8.97,+,0.20,-0.22,0.111483
3774,21819494.0,4274.0,8.84,8.84,8.70,8.75,-,0.22,0.19,0.320000
4645,28184977.0,7204.0,8.75,9.04,8.75,8.94,+,0.19,-0.14,0.134228
5516,25128836.0,5207.0,9.00,9.04,8.80,8.80,-,0.14,0.10,0.022727
6387,11023444.0,2643.0,8.96,9.02,8.88,8.90,+,0.10,0.12,-0.359551
7258,24175523.0,4853.0,8.93,9.06,8.82,9.02,+,0.12,-0.13,-0.687361
8129,14471305.0,3988.0,9.02,9.06,8.87,8.89,-,0.13,0.11,-0.607424


In [21]:
#in order to predict the future price with given input shift upward 1
# df_c = df_c.shift(-1)

In [22]:
#label the output with 5 days after the current day and mark with 0 reprecent loss and 1 with up rising
# df_c['label'] = np.where((df_c['收盤價'].shift(-5)-df_c['收盤價'])/df_c['收盤價']>0,1,0)

In [23]:
del df_c['漲跌(+/-)']
del df_c['漲跌價差']
del df_c['return']

In [24]:
#minmax the value useing MinMaxScaler

min_max_scaler = MinMaxScaler(feature_range=(0, 1))
# data_scaled = min_max_scaler.fit_transform([np.float32(df_c)])
for i in range(len(df_c.columns)-1):
    df_c[df_c.columns[i]] =min_max_scaler.fit_transform(np.array(df_c[df_c.columns[i]]).reshape(-1,1))
# df_c=df_c.fillna(0)
# min_max_scaler1 = MinMaxScaler(feature_range=(-3, 3))
# df_c['label'] =min_max_scaler1.fit_transform(np.array(df_c['label']).reshape(-1,1))

In [25]:
df_c

,成交股數,成交筆數,開盤價,最高價,最低價,收盤價,label
290,0.049967,0.067578,0.112022,0.111585,0.116658,0.112167,0.481100
1161,0.107178,0.121532,0.111680,0.113803,0.117543,0.115105,-0.224719
2032,0.068833,0.080808,0.114242,0.113291,0.116658,0.112859,0.296465
2903,0.077944,0.110923,0.111510,0.115680,0.115950,0.116315,0.111483
3774,0.072128,0.082129,0.112705,0.111756,0.116658,0.112513,0.320000
4645,0.094503,0.142607,0.111168,0.115168,0.117543,0.115797,0.134228
5516,0.083760,0.101387,0.115437,0.115168,0.118428,0.113377,0.022727
6387,0.034179,0.048464,0.114754,0.114827,0.119844,0.115105,-0.359551
7258,0.080409,0.094080,0.114242,0.115509,0.118782,0.117179,-0.687361
8129,0.046298,0.076226,0.115779,0.115509,0.119667,0.114933,-0.607424


In [26]:
#seperate label and input
Y_batch = df_c['label']
del df_c['label']
X_batch = df_c
X_batch = np.array(X_batch)
Y_batch = np.array(Y_batch)

In [467]:
#make categorical
# Y_batch = Y_batch.reshape(1206,1)
# Y_batch1=[]
# for i in range(Y_batch.shape[0]):
#     if Y_batch[i]==1:
#         Y_batch1.append([1])
#     else:
#         Y_batch1.append([0])

In [468]:
# Y_batch1 = np_utils.to_categorical(Y_batch1)

In [469]:
#seperate train and test data 8/2
seperate_rate=0.9
X_batch_train = X_batch[:int(X_batch.shape[0]*seperate_rate)]
X_batch_test = X_batch[int(X_batch.shape[0]*seperate_rate):]
# Y_batch_train = Y_batch1[:int(Y_batch1.shape[0]*seperate_rate)]
# Y_batch_test = Y_batch1[int(Y_batch1.shape[0]*seperate_rate):]
Y_batch_train = Y_batch[:int(Y_batch.shape[0]*seperate_rate)]
Y_batch_test = Y_batch[int(Y_batch.shape[0]*seperate_rate):]

In [470]:
X_batch_train.shape

(1021, 6)

In [471]:
X_batch_test.shape

(114, 6)

In [472]:
X_batch_train = np.delete(X_batch_train,0,axis=0)
X_batch_test = np.delete(X_batch_test,[0,1,2,3],axis=0)
Y_batch_train = np.delete(Y_batch_train,0,axis=0)
Y_batch_test = np.delete(Y_batch_test,[0,1,2,3],axis=0)

In [473]:
#create time sequence input and label
time_steps=5
X_batch_train_time =[]
Y_batch_train_time = []
X_batch_test_time = []
Y_batch_test_time =[]
tlx=[]
tly=[]
for i in range(len(X_batch_train)-time_steps):
    try:
        tlx.append(X_batch_train[i:i+time_steps])
        tly.append(Y_batch_train[i:i+time_steps])
    except:
        pass
    X_batch_train_time.append(tlx)
    tlx=[]
    Y_batch_train_time.append(tly)
    tly = []
tlx1=[]
tly1=[]
for ii in range(len(X_batch_test)-time_steps):
    try:
        tlx1.append(X_batch_test[ii:ii+time_steps])
        tly1.append(Y_batch_test[ii:ii+time_steps])
    except:
        pass
    X_batch_test_time.append(tlx1)
    tlx1=[]
    Y_batch_test_time.append(tly1)
    tly1 = []
X_batch_train_time =np.array(X_batch_train_time)
Y_batch_train_time =np.array(Y_batch_train_time)
X_batch_test_time = np.array(X_batch_test_time)
Y_batch_test_time =np.array(X_batch_test_time)

In [474]:
X_batch_train_time.shape

(1015, 1, 5, 6)

In [475]:
X_batch_train_time = X_batch_train_time.reshape(-1,time_steps,6)
Y_batch_train_time = Y_batch_train_time.reshape(-1,time_steps,1)
X_batch_test_time = X_batch_test_time.reshape(-1,time_steps,6)
Y_batch_test_time = Y_batch_test_time.reshape(-1,time_steps,1)

In [476]:
Y_batch_train_time.shape

(1015, 5, 1)

In [477]:
Y_batch_test_time.shape

(630, 5, 1)

In [478]:
# X_batch_train_time = np.delete(X_batch_train_time,[0,1,2,3],axis=0)
# Y_batch_train_time = np.delete(Y_batch_train_time,[0,1,2,3],axis=0)

In [479]:
Y_batch_train_time.shape

(1015, 5, 1)

In [480]:
#seperate by batch
BATCH_SIZE=5
X_batch_train_time_b=[]
Y_batch_train_time_b=[]
X_batch_test_time_b=[]
Y_batch_test_time_b=[]
for i in range(0,X_batch_train_time.shape[0],BATCH_SIZE):
    temp_list=[]
    temp_list1 = []
    X_batch_train_time_b.append(X_batch_train_time[i:i+BATCH_SIZE])
    Y_batch_train_time_b.append(Y_batch_train_time[i:i+BATCH_SIZE])
    X_batch_test_time_b.append(X_batch_test_time[i:i+BATCH_SIZE])
    Y_batch_test_time_b.append(Y_batch_test_time[i:i+BATCH_SIZE])

In [481]:
X_batch_train_time_b = np.array(X_batch_train_time_b)
Y_batch_train_time_b = np.array(Y_batch_train_time_b)
X_batch_test_time_b = np.array(X_batch_test_time_b)
Y_batch_test_time_b = np.array(Y_batch_test_time_b)

In [482]:
start = 0
BATCH_SIZE=5
TIME_STEPS = 5
BATCH_N = int(X_batch_train_time_b.shape[0]/BATCH_SIZE/TIME_STEPS)
INPUT_SIZE = 6
CELL_SIZE = 3
OUTPUT_SIZE=16

In [483]:
LR = 0.0001

In [484]:
model =Sequential()
model.add(LSTM(
    batch_input_shape=(BATCH_SIZE,TIME_STEPS, INPUT_SIZE),
    output_dim=128,
    return_sequences=True,
    stateful=True,
))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(OUTPUT_SIZE)))
model.add(Dense(64,activation='tanh'))
model.add(Dense(8))
model.add(Dense(1))
adam = Adam(LR)
model.compile(optimizer=adam,
              loss='mse')
# model.load_weights('3029withpercentage.h5', by_name=True)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(batch_input_shape=(5, 5, 6), return_sequences=True, stateful=True, units=128)`
  


In [485]:
cl = []
predl=[]

In [ ]:
epoch = 3001
for epoch in range(epoch):
    for i in range(X_batch_train_time_b.shape[0]):
        X = X_batch_train_time_b[i]
        Y = Y_batch_train_time_b[i]
        cost = model.train_on_batch(X,Y)
        pred = model.predict(X, BATCH_SIZE)
        cl.append(cost)
        
    if epoch % 100  == 0:
        print('cl',np.average(cl))
#         l = model.layers[0].get_weights()
        print('train cost: ', cost,'epoch: ',epoch)
        cl=[]

cl 8.6538
train cost:  0.21585104 epoch:  0
cl 8.632542
train cost:  0.26759452 epoch:  100
cl 8.645388
train cost:  0.2203295 epoch:  200
cl 8.691951
train cost:  0.22433989 epoch:  300
cl 8.557542
train cost:  0.2908329 epoch:  400
cl 8.621979
train cost:  0.19057111 epoch:  500
cl 8.507031
train cost:  0.26119253 epoch:  600
cl 8.403448
train cost:  0.19677183 epoch:  700
cl 8.364069
train cost:  0.73080647 epoch:  800
cl 8.181507
train cost:  0.2827293 epoch:  900
cl 8.265502
train cost:  0.3580439 epoch:  1000
cl 8.339878
train cost:  0.22134893 epoch:  1100
cl 8.16265
train cost:  0.55423295 epoch:  1200
cl 8.035689
train cost:  0.35153198 epoch:  1300
cl 7.7612314
train cost:  0.34272537 epoch:  1400
cl 7.641239
train cost:  0.30350265 epoch:  1500
cl 7.1942425
train cost:  0.37106213 epoch:  1600
cl 6.700353
train cost:  0.36431772 epoch:  1700
cl 6.1270323
train cost:  0.23118325 epoch:  1800
cl 5.603988
train cost:  0.22583668 epoch:  1900
cl 5.453348
train cost:  0.21235791 

In [ ]:
# model.save_weights('3029withpercentage.h5')


In [172]:
X_batch_test = np.delete(X_batch_test,0,axis=0)
Y_batch_test = np.delete(Y_batch_test,0,axis=0)

In [204]:
X_batch_test_time_b=X_batch_test_time_b[0:20]

In [205]:
X_batch_test_time_b[-1].shape

(5, 11, 6)

In [210]:
import sklearn

In [206]:
yhat=[]
for i in range(X_batch_test_time_b.shape[0]):
    print(X_batch_test_time_b[i].shape)
    yhat.append(model.predict_on_batch(X_batch_test_time_b[i]))

(5, 11, 6)
(5, 11, 6)
(5, 11, 6)
(5, 11, 6)
(5, 11, 6)
(5, 11, 6)
(5, 11, 6)
(5, 11, 6)
(5, 11, 6)
(5, 11, 6)
(5, 11, 6)
(5, 11, 6)
(5, 11, 6)
(5, 11, 6)
(5, 11, 6)
(5, 11, 6)
(5, 11, 6)
(5, 11, 6)
(5, 11, 6)
(5, 11, 6)


In [216]:
yhat

[array([[[0.33940035],
         [0.33999252],
         [0.34042376],
         [0.3401445 ],
         [0.34012517],
         [0.34028682],
         [0.34010768],
         [0.3400021 ],
         [0.33972228],
         [0.33998185],
         [0.33978164]],
 
        [[0.34016377],
         [0.34052446],
         [0.34022045],
         [0.34018654],
         [0.34033883],
         [0.3401531 ],
         [0.34004295],
         [0.33976078],
         [0.34001765],
         [0.33981454],
         [0.33976206]],
 
        [[0.34049582],
         [0.34020326],
         [0.34017527],
         [0.34033206],
         [0.34014884],
         [0.3400398 ],
         [0.33975843],
         [0.3400153 ],
         [0.3398131 ],
         [0.33976063],
         [0.339753  ]],
 
        [[0.339584  ],
         [0.33956534],
         [0.33972973],
         [0.33955002],
         [0.33941424],
         [0.33911273],
         [0.33934423],
         [0.339148  ],
         [0.3390714 ],
         [0.33904564],
  

In [215]:
yhat1 = MinMaxScaler.inverse_transform(yhat)

TypeError: inverse_transform() missing 1 required positional argument: 'X'